# Room Table Cleaning

In [2]:
import pandas as pd
import json
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

In [59]:
room_info = pd.read_csv("rooms_100k.csv")
room_info.head()

,uuid,floor_id,data,_fivetran_synced,_fivetran_deleted,work_units,physical_desk_count
0,c3908d85-7f57-45ed-b2c8-d2ba2c25037b,00018f5b-75e0-4f27-a01a-576fd6b9050a,"{""name"":""PRIVATE OFFICE"",""number"":""02-106"",""uu...",2018-11-13 05:13:27.541,NaN,18.0,18.0
1,119e406f-ff59-4019-ad88-f14376e93d57,00018f5b-75e0-4f27-a01a-576fd6b9050a,"{""name"":""PRIVATE OFFICE"",""number"":""02-116"",""uu...",2018-11-13 05:13:27.541,NaN,19.0,19.0
2,a9064324-9c09-426e-998f-bb066aead2e6,00018f5b-75e0-4f27-a01a-576fd6b9050a,"{""name"":""PRIVATE OFFICE"",""number"":""02-110"",""uu...",2018-11-13 05:13:27.541,NaN,24.0,24.0
3,37801461-6d16-4f21-bc72-f1183ba67dd8,00018f5b-75e0-4f27-a01a-576fd6b9050a,"{""name"":""PRIVATE OFFICE"",""number"":""02-109"",""uu...",2018-11-13 05:13:27.541,NaN,31.0,31.0
4,45bc8773-69d1-49a2-8f9a-662c29265fe7,00018f5b-75e0-4f27-a01a-576fd6b9050a,"{""name"":""CONFERENCE ROOM"",""number"":""02A"",""uuid...",2018-11-13 05:13:27.541,NaN,0.0,0.0


In [70]:
n_row = room_info.shape[0]

col_name = list(json.loads(room_info.data.iloc[0]).keys())
stats_col = col_name.copy()

room_data_stats  = pd.DataFrame(columns = stats_col)

for i in range(n_row):
    r_info = json.loads(room_info.data.iloc[i])

    if 'outline' in r_info.keys():
        r_info['outline'] = str(r_info['outline'])
    df = pd.DataFrame(r_info, index = [i])
    room_data_stats = room_data_stats.append(df)

In [67]:
room_data_stats.to_csv("room_data_stats.csv")

In [71]:
room_data_stats.columns

Index(['name', 'number', 'uuid', 'id', 'program_type', 'area', 'level',
       'location', 'outline', 'centroid', 'desk_count', 'physical_desk_count',
       'work_units', 'table_count', 'internal_room_count',
       'extra_furniture_count', 'soft_seating_count',
       'percent_utilizable_area', 'plan_irregularity', 'average_minimum_span',
       'has_window', 'room_boundaries'],
      dtype='object')

In [68]:
# Keep features from original table:
#  ---- 'uuid': unique id of floor plan; 
#  ---- 'property_id': unique id of each building (a property may contain multiple floor plans since multiple-level ); 
#  ---- '_fivetran_synced': created date at system
desired_column = ['uuid', 'floor_id', '_fivetran_synced']
room_basic = room_info[desired_column]
room_basic = room_basic.iloc[0:10000]
# delete 'uuid' in floor_data_stats (?) not clear
# floor_data_stats = floor_data_stats.drop(['uuid'], axis=1)

# combine two tables to get whole information of floor 
if room_data_stats.shape[0] == room_basic.shape[0]:
    room_df = pd.concat([room_basic, room_data_stats], axis=1, join_axes=[room_basic.index])
else:
    room_df = pd.concat([room_basic, room_data_stats], axis=1, join_axes=[room_basic.index])
    print('warning! You may check your table dimension')
print('shape of room_data_stats', room_data_stats.shape)
print('shape of room_basic', room_basic.shape) 
print('shape of room_df(merged)', room_df.shape)



shape of room_data_stats (10000, 24)
shape of room_basic (10000, 3)
shape of room_df(merged) (10000, 27)


In [69]:
room_df.to_csv('room_CLEANED.csv')